# Counterexample X

### Installations

The below will install the necessary packages to run the notebook.

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scipy

The following block imports the relevant modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.integrate import cumulative_trapezoid
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

The below helper functions will be useful later.

In [ ]:
# Computes the integral of f over x
def compute_integral(x_vals, f_vals):
    """
    Given arrays of x-values and the corresponding f(x)-values,
    compute and plot the cumulative integral:
    
        F(x) = ∫ f(t) dt from t=0 to t=x.
    
    Parameters
    ----------
    x_vals : array_like
        Monotonically increasing array of x-values.
    f_vals : array_like
        Corresponding values of f(x).
    """
    # Ensure x_vals and f_vals are numpy arrays
    x_vals = np.asarray(x_vals)
    f_vals = np.asarray(f_vals)
    
    # Compute the cumulative integral using scipy.integrate.cumtrapz
    F_vals = cumulative_trapezoid(f_vals, x_vals, initial=0)

    # Normalize the integral
    F_vals = F_vals 

    # Return result
    return(F_vals)

In [ ]:
def plot_surface_3d(F_vals, x_vals, y_vals=None, downsample=1, title='Surface Plot', z_title='Z', sphere_radius=None):
    """
    Plots a 3D surface for F_vals over the given x and y values.

    Parameters:
    - F_vals: 2D array of values representing F(x, y)
    - x_vals: 1D array for x-axis (or 2d precomputed grid coordinates)
    - y_vals: 1D array for y-axis (or 2d precomputed grid coordinates) (optional; defaults to x_vals)
    - downsample: int factor to reduce resolution (e.g., 2 means every 2nd point)
    - sphere_radius: float, optional — if provided, a translucent sphere of this radius is added
    """

    if y_vals is None:
        y_vals = x_vals

    if y_vals.ndim != x_vals.ndim:
        raise ValueError('x and y must have same number of dimensions.')

    # Downsample data if needed
    if downsample > 1:
        F_vals = F_vals[::downsample, ::downsample]

        if x_vals.ndim == 1:
            x_vals = x_vals[::downsample]
            y_vals = y_vals[::downsample]
        else:
            x_vals = x_vals[::downsample, ::downsample]
            y_vals = y_vals[::downsample, ::downsample]

    # Create 2D grid (X, Y)
    if x_vals.ndim == 1:
        X, Y = np.meshgrid(x_vals, y_vals, indexing='xy')
    else:
        X = x_vals
        Y = y_vals

    # Create surface plot list
    surfaces = [
        go.Surface(
            x=X,
            y=Y,
            z=F_vals,
            colorscale='Viridis',
            name='Main Surface'
        )
    ]

    # Optionally add a translucent sphere
    if sphere_radius is not None:
        u = np.linspace(0, np.pi, 50)
        v = np.linspace(0, 2 * np.pi, 50)
        U, V = np.meshgrid(u, v)
        Xs = sphere_radius * np.sin(U) * np.cos(V)
        Ys = sphere_radius * np.sin(U) * np.sin(V)
        Zs = sphere_radius * np.cos(U)

        surfaces.append(
            go.Surface(
                x=Xs,
                y=Ys,
                z=Zs,
                opacity=0.3,
                showscale=False,
                colorscale='Blues',
                name=f'Sphere r={sphere_radius}'
            )
        )

    # Create figure
    fig = go.Figure(data=surfaces)

    # Layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title=z_title
        ),
        autosize=True,
        width=800,
        height=700
    )

    fig.show()


### Introduction

In this notebook, we shall construct a $C^1$ function $f:\mathbb{R}^3\rightarrow \mathbb{R}$ with the following properties:

 - $f(0,0,0)=0$.
 - The only critical point of $f$ occurs at the origin.
 - for every $\epsilon > 0$, the zero level set $f^{-1}(0)$ intersects the boundary of the ball $\partial B_\epsilon \subset \mathbb{R}^3$ tangentially.

To achieve this goal, we construct the function in two steps, with intermediate and final expressions given in spherical coordinates $(r, \theta, \psi)$, where $r \in [0,\infty)$, $\theta \in [0,\pi]$, and $\psi \in [0,2\pi)$:

- **Step 1:** Define a $C^1$ function $H : [0,\pi] \times [0, 2\pi) \rightarrow [0,\infty)$ such that the surface given by $r = H(\theta, \psi)$ is tangent to every ball of radius less than or equal to 1.

- **Step 2:** Define a $C^1$ function $G : [0,\pi] \times [0, 2\pi) \rightarrow [0,\infty)$ which acts as a "nice" lower bound to $H$.

- **Step 3:** Construct a $C^1$ function $f : [0,\infty) \times [0,\pi] \times [0,2\pi) \rightarrow \mathbb{R}$ with zero level set given by
  
  $$f^{-1}(0) = \{ (r, \theta, \psi) \mid r = H(\theta, \psi) \},$$
  zero gradient at the origin, and non-vanishing gradient everywhere else within a neighbourhood of the origin.

We treat $[0,\infty) \times [0,\pi] \times [0,2\pi)$ as spherical coordinates on $\mathbb{R}^3$, with the usual identification of $\psi = 0$ and $\psi = 2\pi$, and the azimuthal degeneracy at the poles $\theta = 0$ and $\theta = \pi$.



### Step 1: Constructing $H$

To construct $H$, we begin in Cartesian coordinates $(x, y, z)$ by defining an intermediate function $H^\ast : \mathbb{R}^2 \to \mathbb{R}$ such that, for every $z \in [0,1]$, there exists a point $(x, y) \in [0,1]^2$ with $H^\ast(x, y) = z$ and $\nabla H^\ast(x, y) = 0$. This construction follows the approach of [Grinberg (2018)](link), but we provide explicit equations throughout.


To be specific, we construct $H$ in several steps:

1. **One-dimensional slice:** We begin by defining a function $\tilde{H} : [0,1] \to [0,1]$ that has a critical point at every level $c \in C$, where $C$ denotes the middle-thirds Cantor set.

2. **Symmetrization:** We extend this function to $[-1,1]$ by requiring it to be even.

3. **Product extension:** Identifying $[-1,1]$ with $[-1,1] \times \{0\}$, we extend the domain to $[-1,1]^2$ by defining  
   $$
   H^\ast(x, y) = \frac{1}{2}(\tilde{H}(x) + \tilde{H}(y)).
   $$

4. **Global extension:** Extending the definition of $H^\ast$ on $[-1,1]^2$ from above, we now define $H^\ast(x, y) = 1$ for all $(x, y) \in \mathbb{R}^2 \setminus [-2,2]^2$, and use a $C^1$ transition function to smoothly interpolate over the rectangular region $[-2,2]^2 \setminus [-1,1]^2$, resulting in a globally $C^1$ function.

5. **Mapping to Spherical:** We next define $H$ in terms of spherical coordinates $(\theta,\psi)$ as $H(\theta,\psi):=H^\ast(\frac{\pi}{2k}(\theta+k),\frac{\pi}{k}(\psi+k))$ for some suitably large $k$.

#### One Dimensional Slice

To construct the 1 dimensional function with critical points at every level $c \in C$, we shall use a fractal construction for the derivative of $\tilde{H}$ which involves carefully placing and summing spikes of precomputed areas.

We start by letting $n\in\mathbb{N}_{>0}$ and $k \in \{0, 1, \dots, 2^n - 1\}$, and write the binary expansion of $k$ as:

\begin{equation}\nonumber
k = \sum_{j=1}^n r_j \cdot 2^{n-j}, \quad \text{where } r_j \in \{0,1\}.
\end{equation}
We then define:
\begin{equation}\nonumber
a_{n,k} := \sum_{j=1}^n r_j \cdot \frac{3}{5} \cdot \left( \frac{2}{5} \right)^{j-1},\quad\text{and}\quad b_{n,k} := a_{n,k} + \left( \frac{2}{5} \right)^n.
\end{equation}


Next, we define the following 'spike' function for $x \in [0,1]$, $n \in \mathbb{N}_{>0}$, and $k \in \{0, \dots, 2^{n-1} - 1\}$:

$$
\text{Spike}_{k,n}(x) = 
\begin{cases}
    0 & \text{if } x \leq \frac{1}{2}(a_{n,k} + b_{n,k} - w_n), \\
    2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k}), \\
    h_n - 2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k}) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k}) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n), \\
    0 & \text{if } x \geq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n).
\end{cases}
$$

where the height and width are given by:

$$
h_n := 4 \left( \frac{5}{6} \right)^{n+1}, \qquad w_n := \frac{2}{h_n \cdot 3^{n+1}}.
$$

To give some intuition as to what this looks like, we provide a function below:

In [ ]:
import numpy as np

def spike(x, n, k):
    """
    Vectorized version of the spike function Spike_{k,n}(x) for NumPy arrays.
    
    Parameters:
        x : np.ndarray
            Input values in [0, 1]
        n : int
            Scale level (must be >= 1)
        k : int
            Index in {0, ..., 2^n - 1}
            
    Returns:
        np.ndarray : Values of the spike function at x
    """
    x = np.asarray(x)
    
    # Convert k to binary vector r = [r1, ..., rn]
    r = [(k >> (n - j - 1)) & 1 for j in range(n)]
    
    # Compute a_{n,k}
    a_nk = sum(r_j * (3/5) * (2/5)**j for j, r_j in enumerate(r))
    
    # Compute b_{n,k}
    b_nk = a_nk + (2/5)**n
    
    # Compute height and width
    h_n = 4 * (5/6)**(n + 1)
    w_n = 2 / (h_n * 3**(n + 1))
    
    # Midpoint and boundaries
    mid = 0.5 * (a_nk + b_nk)
    left = mid - 0.5 * w_n
    right = mid + 0.5 * w_n

    # Initialize result
    y = np.zeros_like(x)

    # Linear rising edge
    mask_rise = (x > left) & (x <= mid)
    y[mask_rise] = 2 * h_n / w_n * (x[mask_rise] - left)

    # Linear falling edge
    mask_fall = (x > mid) & (x < right)
    y[mask_fall] = h_n - 2 * h_n / w_n * (x[mask_fall] - mid)

    # Peak (only applies if a value matches mid exactly, which is rare)
    y[np.isclose(x, mid)] = h_n

    return y

And an example plot:

In [ ]:
import matplotlib.pyplot as plt

n = 0
k = 0
x_vals = np.linspace(0, 1, 1000)
y_vals = spike(x_vals, n, k)

plt.plot(x_vals, y_vals)
plt.title(f"Spike function for n={n}, k={k}")
plt.xlabel("x")
plt.ylabel(f"Spike_{{{k},{n}}}(x)")
plt.grid(True)
plt.show()

The 'spike' function creates a triangular signal, with carefully chosen width and position, which encloses an area of exactly $(\frac{2}{3})^{n+1}$.

The width and position of the spikes have been carefully chosen so that we may define a function $h$ as follows:

\begin{equation}\nonumber 
    h(x)= \sum_{n=0}^{\infty}\sum_{i=0}^{2^n-1} \text{Spike}_{k,n}(x).
\end{equation}

In [ ]:
def compute_h(x, N_max=5):
    """
    Compute an approximation of h(x) = sum_{n=0}^{∞} sum_{k=0}^{2^n - 1} Spike_{k,n}(x)
    using a finite sum up to N_max.
    
    Parameters:
        x : np.ndarray
            Array of input values in [0,1].
        N_max : int
            Maximum value of n to include in the approximation (n=0 to N_max).
    
    Returns:
        np.ndarray : Approximated h(x) values.
    """
    x = np.asarray(x)
    h_vals = np.zeros_like(x)

    for n in range(N_max + 1):
        for k in range(2**n):
            h_vals += spike(x, n, k)
    
    return h_vals

In [ ]:
x_vals = np.linspace(0, 1, 1000)
h_vals = compute_h(x_vals, N_max=6)

plt.plot(x_vals, h_vals)
plt.title("Approximation of h(x) with N_max=6")
plt.xlabel("x")
plt.ylabel("h(x)")
plt.grid(True)
plt.show()

The function $h$ is a fractal function consisting of infinitely many non-overlapping spikes. Due to the choices of $h_n$, $w_n$, $a_{n,k}$, and $b_{n,k}$ used in the construction, the spikes become shorter and narrower as $n$ increases. As a result, it is straightforward to verify that $h$ is continuous.

The construction begins with a single spike of area $\frac{2}{3}$, followed by 2 spikes of area $\left(\frac{2}{3}\right)^2$, then 4 spikes of area $\left(\frac{2}{3}\right)^3$, and so on. A simple calculation shows that the total area under the graph of $h$ over $[0,1]$ is exactly 1.

This reasoning can be extended further. As every $c \in C$ can be expressed as a base 3 expansion using only $0$s and $2$s, or equivalently as a linear combination of powers of $\frac{2}{3}$, it easily follows that, for all $c\in C$, there exists an $x \in [0,1]$ such that the cumulative area under $h$ from $0$ to $x$ is exactly $c$. This defines a bijection between $C$ and the set of zeros of $h$, where $h(x) = 0$ if and only if the cumulative area up to $x$ equals $c$.


Noting this this, we now define $\tilde{H}:[0,1]\rightarrow [0,1]$ as follows:

\begin{equation}\nonumber
    \tilde{H}(x):=\int_{t=0}^x h(t)dt.
\end{equation}


In [ ]:
def compute_H_tilde(x_vals, normalise=True):
    
    # Separate into negative and non-negative parts
    x_neg = x_vals[x_vals < 0]
    x_pos = x_vals[x_vals >= 0]

    # Prepare result array
    H_tilde_vals = np.empty_like(x_vals, dtype=float)

    # Compute H for negative values
    if len(x_neg) > 0:

        # Flip sign and reverse order
        x_neg_flipped = -x_neg[::-1]  

        # Compute h values for negative
        h_neg = compute_h(x_neg_flipped, N_max=6)

        # Compute integral for negative
        H_tilde_neg = compute_integral(x_neg_flipped, h_neg)
        
        # Reverse to match original x_neg order
        H_tilde_neg = H_tilde_neg[::-1]  

        # Insert back into array
        H_tilde_vals[:len(H_tilde_neg)] = H_tilde_neg

    # Compute H for positive values
    if len(x_pos) > 0:
        
        # Compute h values for positive
        h_pos = compute_h(x_pos, N_max=6)
        
        # Compute integral for positive
        H_tilde_pos = compute_integral(x_pos, h_pos)

        # Insert back into array
        H_tilde_vals[-len(H_tilde_pos):] = H_tilde_pos

    # Normalise entire result if requested
    if normalise:
        H_tilde_vals = H_tilde_vals / H_tilde_vals[-1]

    # Return results
    return H_tilde_vals


In [ ]:
x_vals = np.linspace(0, 1, 1000)
H_tilde_vals = compute_H_tilde(x_vals)

plt.plot(x_vals, H_tilde_vals)
plt.title(r"Approximation of $\tilde{H}$(x)")
plt.xlabel("x")
plt.ylabel(r"$\tilde{H}(x)$")
plt.grid(True)
plt.show()

#### Symmetrization

We then extend this definition to $[-1,1]$ by taking $\tilde{H}(x):=\tilde{H}(-x)$ for $x<0$. Note that, as $\tilde{H}'(0)=h(0)=0$, $\tilde{H}$ is continuously differentiable across the whole of $[-1,1]$.

In [ ]:
x_vals = np.linspace(-1, 1, 1000)
H_tilde_vals = compute_H_tilde(x_vals)

plt.plot(x_vals, H_tilde_vals)
plt.title(r"Approximation of $\tilde{H}$(x)")
plt.xlabel("x")
plt.ylabel(r"$\tilde{H}(x)$")
plt.grid(True)
plt.show()

By construction, $\tilde{H}$ has critical points at every level $c\in C$. 


#### Product Extension

Next, we define a two-dimensional function $H^\ast:[-1,1]^2\rightarrow [-1,1]$ by $H^\ast(x,y):=\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$.

In [ ]:
# Function to compute H
def compute_H_star(x_vals,y_vals=None):

    # Assume we are using the same axes for x and y if not given different
    if y_vals is None:
        y_vals = x_vals
        
    # Compute H_star values for one slice
    H_star_vals_slice = compute_H_tilde(x_vals)
    
    # Create 2D grid (X, Y) and compute H(|x|) + H(|y|)
    X, Y = np.meshgrid(x_vals, x_vals, indexing='xy')
    H_star_vals = np.zeros_like(X)
    
    # Compute number of points
    num_points = len(x_vals)
    
    # Compute H_star for whole surface
    for i in range(num_points):
        for j in range(num_points):
            H_star_vals[i, j] = (H_star_vals_slice[i] + H_star_vals_slice[j])/2

    return(H_star_vals)

# Compute H_star_vals
H_star_vals = compute_H_star(x_vals)

In [ ]:
# Plot surface
plot_surface_3d(H_star_vals, x_vals, downsample=2, title="3D Surface With Critical Values [0,1]", z_title = "H*(x, y) = H\u0303(x) + H\u0303(y)")

Through exploiting a well known function of the Cantor set; $C+C=[0,2]$, it can be seen that $H$ has a critical value at every level $z\in[0,1]$.

#### Global Extension

Next, we extend $H^\ast$ to the entirety of $\mathbb{R}^2$. This is achieved using the $C^1$ transition function $s(x)$ defined by:

$$
s(x):=\begin{cases}
0 & x \leq 0, \\
\frac{x^2}{x^2+(1-x)^2} & 0 \leq x \leq 1, \\
1 & 1 \leq x. \\
\end{cases}
$$

We then extend $\tilde{H}$ to $\mathbb{R}$ by letting $\tilde{H}(x)=1$ for $x \not\in[-1,1]^2$ and extend the definition of $H^\ast$ as follows:

$$H^\ast(x,y):=s\bigg(d\big((x,y),[-1,1]^2\big)\bigg)+\bigg(1-s\bigg(d\big((x,y),[-1,1]^2\big)\bigg)\bigg)\left(\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))\right),$$

where $d(p,A):=\inf_{a\in A}|p-a|$ represents the Euclidean distance from the point $p$ to the set $A$. 

The above notation is dense, but much easier to understand when visualised. When $d((x,y),[-1,1]^2)\geq 1$, we see that $H^\ast(x,y)=1$, and when $(x,y)\in [-1,1]^2$ we have that $d((x,y),[-1,1]^2)=0$ and thus $H^\ast(x,y)=\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$, as before. In the intermediate region, the function smoothly transitions from $\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$ to $1$.

In [ ]:
def extend_F(F_vals, c_vals, x_grid, y_grid, epsilon=1.0):
    """
    Constructs new_vals with correct epsilon-thickening logic:
      - new_vals = F_vals inside [-1, 1]^2
      - new_vals = T in thickened shell (distance <= epsilon, not box)
      - new_vals = k_vals outside the thickened region

    Parameters:
    - F_vals: np.ndarray, values of H(x, y)
    - c_vals: np.ndarray, values of k(x, y)
    - x_grid, y_grid: coordinate grids matching H_vals shape
    - epsilon: thickness parameter for blending

    Returns:
    - new_vals: np.ndarray, constructed values
    """

    new_vals = np.zeros_like(F_vals)

    # Step 1: Compute distance from each point to [-1,1]^2 box
    dx = np.maximum(np.abs(x_grid) - 1, 0)
    dy = np.maximum(np.abs(y_grid) - 1, 0)
    dist = np.sqrt(dx**2 + dy**2)

    # Step 2: Define regions
    in_core = (np.abs(x_grid) <= 1) & (np.abs(y_grid) <= 1)
    in_thickened = (dist <= epsilon) & (~in_core)
    outside = dist > epsilon

    # Step 3: Compute blending weights in shell region
    d = dist
    denom = d**2 + (1 - d)**2
    denom[denom == 0] = 1  # Avoid division by zero
    w = d**2 / denom

    # Step 4: Assign values
    new_vals[in_core] = F_vals[in_core]
    new_vals[in_thickened] = w[in_thickened] * c_vals[in_thickened] + (1 - w[in_thickened]) * F_vals[in_thickened]
    new_vals[outside] = c_vals[outside]

    return new_vals

In [ ]:
# x values
x_vals = np.linspace(-6,6,1000)
y_vals = np.linspace(-6,6,1000)

# Compute H_star_vals
H_star_vals = compute_H_star(x_vals,y_vals)

# Constant values
c_vals = np.ones(H_star_vals.shape)

# Generate 2D grids from x_vals
X, Y = np.meshgrid(x_vals, y_vals)

# Extend H*
H_star_vals = extend_F(H_star_vals, c_vals, X, Y, epsilon=1.0)

# Extend H star
plot_surface_3d(H_star_vals, x_vals, downsample=2, title="3D Surface With Critical Values [0,1]", z_title="H*(x, y) = H\u0303(x) + H\u0303(y)")

#### Mapping to Spherical

We next define $H$ in spherical coordinates $(\theta,\psi)$ by
$$
H(\theta,\psi) := H^*\left(\frac{\pi}{2k}(\theta + k),\, \frac{\pi}{k}(\psi + k)\right)
$$
for some suitably large integer $k$. Geometrically, this corresponds to embedding a small patch of the plane, containing the critical structure of $H^*$, onto the surface of a sphere of radius 1. Intuitively, one can think of this as removing a small 'panel' from the sphere and replacing it with a rescaled and smoothly curved version of $H^*$ that conforms to the spherical geometry. 

The below plot illustrates the surface $r=H(\theta,\psi)$, which by construction is tangent to every sphere with radius in $[0,1]$. MARKER add back in 2d spherical


In [ ]:
# Compute theta and psi
theta_vals = np.linspace(0,1,len(x_vals))*np.pi#(x_vals/2 + 1) * (np.pi/4)     # [π/4, 3π/4]
psi_vals = np.linspace(0,2,len(x_vals))*np.pi           # [π/2, 3π/2]

# Construct grid of (theta,psi) values
Theta, Psi = np.meshgrid(theta_vals, psi_vals, indexing='ij')  # shape (N, N)

# Compute R values on this grid
R = H_star_vals 

# Construct coordinates for visualisation
X = R * np.sin(Theta) * np.cos(Psi)
Y = R * np.sin(Theta) * np.sin(Psi)
Z = R * np.cos(Theta)

# Plot surface, sphere added to demonstrate tangency
plot_surface_3d(Z, X, Y, downsample=2,sphere_radius=None)

### Step 2: Constructing $G$

Next, we construct a $C^1$ function $G^* : \mathbb{R}^2 \to \mathbb{R}$ that acts as a lower bound for $H^*$ and satisfies the following properties:

- $G^*(x, y) \leq H^*(x, y)$ for all $(x, y) \in \mathbb{R}^2$, with equality if and only if $(x, y) = (0, 0)$.
- $\nabla G^*(x, y) \neq 0$ for all $(x, y) \in [-1,1]^2 \setminus P$, where $P = \{(-1, -1), (-1,1), (1,-1), (1,1), (0,0)\}$.
- $\nabla G^*(x, y) = 0$ outside of $[-1,1]^2 \setminus P$.
- The function $\frac{G^*}{H^*}$ is $C^1$, with $\nabla \left( \frac{G^*}{H^*} \right)(0,0) = 0$.


To construct such a function, we adopt a similar approach to constructing $H^\ast$.


1. **One-dimensional slice:** We begin by defining a function $\tilde{G} : [0,1] \to [0,1]$ that has positive gradient on $(0,1)$ and zero gradient on $\{0,1\}$ satisfying $\tilde{G}\leq \tilde{H}$ with equality only at $x=0$ and $x=1$. We also ensure that $\frac{\tilde{G}}{\tilde{H}}$ is differentiable at the origin with gradient zero.

2. **Symmetrization:** We extend this function to $[-1,1]$ by requiring it to be even.

3. **Product extension:** Identifying $[-1,1]$ with $[-1,1] \times \{0\}$, we extend the domain to $[-1,1]^2$ by defining  
   $$
   G^\ast(x, y) = \frac{1}{2}(\tilde{G}(x) + \tilde{G}(y)).
   $$

4. **Global extension:** As with $H^\ast$, we extend the definition of $G^\ast$ on $[-1,1]^2$. Specifically, we define $G^\ast(x, y) = \frac{1}{2}$ for all $(x, y) \in \mathbb{R}^2 \setminus [-2,2]^2$, and use a $C^1$ transition function to smoothly interpolate over the rectangular region $[-2,2]^2 \setminus [-1,1]^2$, resulting in a globally $C^1$ function.

#### One-dimensional slice

Analogously to the construction of $h$, we begin by designing a function $g:[0,1]\rightarrow [0,1]$. This time, instead of summing carefully-placed non-overlapping spikes, we sum carefully-placed 'ramp' functions. These ramps increase linearly from zero to a fixed height over an interval $[a_n, b_n]$, and remain constant thereafter.

For each $n \in \mathbb{N}_{>0}$, define:
$$
a_n := \left( \frac{2}{5} \right)^n, \qquad b_n := \frac{3}{5} \cdot \left( \frac{2}{5} \right)^{n-1}
$$

Let the ramp height be given by:
$$
h_n := \frac{1}{2} \cdot \frac{1}{3^n(1 - b_n) \cdot n}
$$

We define the individual ramp function $\text{Ramp}_n(x)$ as:
$$
\text{Ramp}_n(x) := 
\begin{cases}
0 & \text{if } x < a_n \\
h_n \cdot \dfrac{x - a_n}{b_n - a_n} & \text{if } a_n \leq x < b_n \\
h_n & \text{if } x \geq b_n
\end{cases}
$$

Then the full function $g$ is given by:
$$
g(x) := \sum_{n=1}^{\infty} \text{Ramp}_n(x)
$$

This construction mirrors the spike-based summation used for $h(x)$, but instead of adding $2^n$ spikes of cumulative area $(\frac{2}{3})^{n+1}$ in each iteration, a single ramp of cumulative area less than $\frac{1}{n}(\frac{2}{3})^{n+1}$ is added. 


Below is a plot of a single ramp.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def ramp(x_vals, n):
    a_n = (2 / 5) ** n
    b_n = (3 / 5) * (2 / 5) ** (n - 1)
    h_n = (1 / 2) * (1 / 3) ** n / (1 - b_n) / n

    ramp_vals = np.zeros_like(x_vals)
    ramp_mask = (x_vals >= a_n) & (x_vals < b_n)
    ramp_vals[ramp_mask] = h_n * (x_vals[ramp_mask] - a_n) / (b_n - a_n)
    ramp_vals[x_vals >= b_n] = h_n
    return ramp_vals

x_vals = np.linspace(0, 1, 1000)
n = 1
y_vals = ramp(x_vals, n)

plt.plot(x_vals, y_vals)
plt.title(f"Ramp function for n={n}")
plt.xlabel("x")
plt.ylabel(f"Ramp_{{{n}}}(x)")
plt.grid(True)
plt.show()


Below is a plot of $g$.

In [ ]:
import numpy as np

def compute_g(x_vals, N_max):
    x_vals = np.array(x_vals)
    total = np.zeros_like(x_vals)

    for i in range(1, N_max + 1):
        a_n = (2 / 5) ** i
        b_n = (3 / 5) * (2 / 5) ** (i - 1)
        h_n = (1 / 2) * (1 / 3) ** i / (1 - b_n) / i

        g_n = np.zeros_like(x_vals)

        # Ramp section: (a_n, 0) to (b_n, h_n)
        ramp_mask = (x_vals >= a_n) & (x_vals < b_n)
        g_n[ramp_mask] = h_n * (x_vals[ramp_mask] - a_n) / (b_n - a_n)

        # Horizontal section: (b_n, h_n) to (1, h_n)
        horiz_mask = (x_vals >= b_n) & (x_vals <= 1)
        g_n[horiz_mask] = h_n

        total += g_n

    return total


In [ ]:
import matplotlib.pyplot as plt

x_vals = np.linspace(0, 1, 1000)
g_vals = compute_g(x_vals, N_max=100)

plt.plot(x_vals, g_vals)
plt.title("g(x) for n=10")
plt.xlabel("x")
plt.ylabel("g(x)")
plt.grid(True)
plt.show()


For reference, we show $h$ and $g$ on the same plot below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example x range
x_vals = np.linspace(0, 1, 1000)

# Compute both functions
h_vals = compute_h(x_vals, N_max=10)  
g_vals = compute_g(x_vals, N_max=100)  
          

# Plot both
plt.plot(x_vals, h_vals, label="h(x)", linestyle='--')   # Dashed line for f
plt.plot(x_vals, g_vals, label="g(x)", linewidth=2)      # Solid line for g

plt.title("Comparison of h(x) and g(x)")
plt.xlabel("x")
plt.ylabel("Function value")
plt.legend()
plt.grid(True)
plt.show()

As before, we now define a function $\tilde{G}$ by integrating $g$. However, unlike with $\tilde{H}$, we also multiply by an additional factor of $\tilde{H}(x)$. This will be of use when computing the gradient of $\frac{\tilde{G}}{\tilde{H}}$ shortly. 


\begin{equation}\nonumber
    \tilde{G}(x):=\tilde{H}(x)\int_{t=0}^x g(t)dt.
\end{equation}


In [ ]:
def compute_G_tilde(x_vals, normalise=False):
    
    # Separate into negative and non-negative parts
    x_neg = x_vals[x_vals < 0]
    x_pos = x_vals[x_vals >= 0]

    # Prepare result array
    G_tilde_vals = np.empty_like(x_vals, dtype=float)

    # Compute H for negative values
    if len(x_neg) > 0:

        # Flip sign and reverse order
        x_neg_flipped = -x_neg[::-1]  

        # Compute h values for negative
        g_neg = compute_g(x_neg_flipped, N_max=6)
        H_tilde_neg = compute_H_tilde(x_neg_flipped)

        # Compute integral for negative
        G_tilde_neg = H_tilde_neg*compute_integral(x_neg_flipped, g_neg)
        
        # Reverse to match original x_neg order
        G_tilde_neg = G_tilde_neg[::-1]  

        # Insert back into array
        G_tilde_vals[:len(G_tilde_neg)] = G_tilde_neg

    # Compute H for positive values
    if len(x_pos) > 0:
        
        # Compute h values for positive
        g_pos = compute_g(x_pos, N_max=6)
        H_tilde_pos = compute_H_tilde(x_pos)
        
        # Compute integral for positive
        G_tilde_pos = H_tilde_pos*compute_integral(x_pos, g_pos)

        # Insert back into array
        G_tilde_vals[-len(G_tilde_pos):] = G_tilde_pos

    # Normalise entire result if requested
    if normalise:
        G_tilde_vals = G_tilde_vals / G_tilde_vals[-1]

    # Return results
    return G_tilde_vals

Below is a plot of $\tilde{H}$ and $\tilde{G}$ for comparison.

In [ ]:
# Create x values
x_vals = np.linspace(0,1,1000)

# Compute H tilde and G tilde
H_tilde_vals = compute_H_tilde(x_vals)
G_tilde_vals = compute_G_tilde(x_vals)

# Plot both
plt.plot(x_vals, H_tilde_vals, label=r"$\tilde{H}$(x)", linestyle='--')   # Dashed line for f
plt.plot(x_vals, G_tilde_vals, label=r"$\tilde{G}$(x)", linewidth=2)      # Solid line for g

plt.title(r"Comparison of $\tilde{H}$(x) and $\tilde{G}$(x)")
plt.xlabel("x")
plt.ylabel("Function value")
plt.legend()
plt.grid(True)
plt.show()

At this point, it is worth noting some properties of $\tilde{G}$ which follow from the above construction. We have that:

 1. By the product rule, for $x>0$, we have:
     $$\tilde{G}'(x) =  h(x)\cdot \bigg(\int_{0}^x g(t)dt\bigg) + \tilde{H}(x) \cdot g(x) > 0,$$
    as all terms in the above expression are non-negative, with $\tilde{H}(x)$ and $g(x)$ being strictly positive.
 2. Similarly, noting $h(0)=g(0)=0$, we have $\tilde{G}'(0)=0$.
 3. The following limit holds:
     $$\lim_{x\rightarrow 0}\bigg(\frac{\tilde{G}}{\tilde{H}}\bigg)(x)=\lim_{x\rightarrow 0}\int_{t=0}^xg(t)dt=0.$$
 4. For $x\neq 0$ the gradient of $\frac{\tilde{G}}{\tilde{H}}$ is given by:
    $$\bigg(\frac{\tilde{G}}{\tilde{H}}\bigg)'(x)=\frac{\partial}{\partial x}\bigg(\int_{t=0}^xg(t)dt\bigg)=g(x)>0.$$
    The discontinuity at $x=0$ is easily seen to be removable and thus $(\frac{\tilde{G}}{\tilde{H}})'(0)=0$.

Before moving on, we must make one final adjustment to $\tilde{G}$. For what follows we shall need $\tilde{G}$ to have gradient 1 at $x=1$. To do so, we use again the $C^1$ transition function $s:\mathbb{R}\rightarrow \mathbb{R}$ to transition between $\tilde{G}$ on $[0,0.5]$ to the constant function with value $\frac{1}{2}$ on $[1,\infty)$. In a slight abuse of notation, the resultant function is what we shall refer to as $\tilde{G}$ going forwards.

In [ ]:
# Smooth transition function
def s(x):
    x = np.asarray(x)
    s_vals = np.zeros_like(x)

    # For 0 < x < 1
    mask_middle = (x > 0) & (x < 1)
    x_mid = x[mask_middle]
    s_vals[mask_middle] = x_mid**2 / (x_mid**2 + (1 - x_mid)**2)

    # For x >= 1
    mask_high = x >= 1
    s_vals[mask_high] = 1

    # x <= 0 stays 0 by default
    return s_vals

def compute_G_tilde_with_transition(x_vals, a=0.5, c=0.5):

    # Handle negative values
    if np.any(x_vals<0):

        # Seperate negatives and positives
        x_vals_neg = x_vals[x_vals<0]
        x_vals_pos = x_vals[x_vals>=0]

        # Reverse order and sign of negative
        x_vals_neg = -x_vals_neg[::-1]

        # Compute seperately for each
        G_tilde_vals_neg = compute_G_tilde_with_transition(x_vals_neg, a=0.5, c=0.5)[::-1]
        G_tilde_vals_pos = compute_G_tilde_with_transition(x_vals_pos, a=0.5, c=0.5)

        # Concetenate
        G_tilde_vals = np.concatenate((G_tilde_vals_neg,G_tilde_vals_pos),axis=0)
        return(G_tilde_vals)
        
    # Compute G tilde
    G_tilde_vals = compute_G_tilde(x_vals)
    
    # Compute transformed argument to s
    s_input = (x_vals - 1) / a + 1
    s_vals = s(s_input)
    
    # Now compute N(x)
    G_tilde_vals = s_vals * c + (1 - s_vals) * G_tilde_vals
    return G_tilde_vals

# Compute G tilde values
G_tilde_vals = compute_G_tilde_with_transition(x_vals)

In [ ]:
# Plot both
plt.plot(x_vals, H_tilde_vals, label=r"$\tilde{H}$(x)", linestyle='--')   # Dashed line for f
plt.plot(x_vals, G_tilde_vals, label=r"$\tilde{G}$(x)", linewidth=2)      # Solid line for g

plt.title(r"Comparison of $\tilde{H}$(x) and $\tilde{G}$(x)")
plt.xlabel("x")
plt.ylabel("Function value")
plt.legend()
plt.grid(True)
plt.show()

It is easily seen that this final definition of $\tilde{G}$ still satisfies properties 2 and 3 listed above. A straightforward computation also shows that property 1 continues to hold, except at the point $x = 1$, where we now have $\tilde{G}'(1) = 0$ instead of $\tilde{G}'(1) > 0$.

The explicit form of the derivative given in property 4 is no longer valid globally, but it remains correct for $x < 0.5$, which in particular implies that
$$
\left( \frac{\tilde{G}}{\tilde{H}} \right)'(0) = 0.
$$

Using the product rule and the fact that $g(x) < 0.5$ for all $x \in [0,1]$, it can further be shown that
$$
\left( \frac{\tilde{G}}{\tilde{H}} \right)'(x) > 0 \quad \text{for all } x \in (0,1), \quad \text{and} \quad \left( \frac{\tilde{G}}{\tilde{H}} \right)'(1) = 0.
$$


#### Symmetrization

As we did with $\tilde{H}$, we next extend $\tilde{G}$ to $[-1,1]$ by assuming that it is an even function. Note that by the above, both $\tilde{G}$ and $\frac{\tilde{G}}{\tilde{H}}$ have gradient zero at the origin and are thus $C^1$ on the extended domain.

In [ ]:
# Create x values
x_vals = np.linspace(-1,1,1000)

# Compute H tilde and G tilde
H_tilde_vals = compute_H_tilde(x_vals)
G_tilde_vals = compute_G_tilde_with_transition(x_vals)

# Plot both
plt.plot(x_vals, H_tilde_vals, label=r"$\tilde{H}$(x)", linestyle='--')   # Dashed line for f
plt.plot(x_vals, G_tilde_vals, label=r"$\tilde{G}$(x)", linewidth=2)      # Solid line for g

plt.title(r"Comparison of $\tilde{H}$(x) and $\tilde{G}$(x)")
plt.xlabel("x")
plt.ylabel("Function value")
plt.legend()
plt.grid(True)
plt.show()

### Step 3: Constructing $f$

We now construct a function $f : [0,\infty) \times [0,\pi] \times [0,2\pi) \rightarrow \mathbb{R}$ whose zero level set corresponds to the surface defined by $H$, that is,
$$
f^{-1}(0) = \{ (r, \theta, \psi) \mid r = H(\theta, \psi) \},
$$
such that $f$ has vanishing gradient at the origin, and non-vanishing gradient in a neighborhood around the origin, excluding the origin itself.
